In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Replace with your actual session cookie from Chrome
SESSION_COOKIE = "17f98ed892c219e814ccc434836ee213"

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Cookie": f"sid={SESSION_COOKIE}"  # replace key name if it's different
}

def fetch_component_materials(component_id: int):
    url = f"https://www.bauteileditor.de/elements/#!/elements/{component_id}?tab=general&_isBaseReq=true"
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch component {component_id}")
    soup = BeautifulSoup(response.text, "html.parser")

    materials = []
    for li in soup.select("#element-composite ol.sortable li .element-selector"):
        material = li.text.strip()
        if material:
            materials.append(material)

    return materials

# Collect data
component_data = []
for cid in range(558048, 558050):  # <-- Adjust range as needed
    try:
        materials = fetch_component_materials(cid)
        if materials:
            component_data.append({
                "component_id": cid,
                "materials": materials,
                "material_count": len(materials)
            })
            print(f"✔️ {cid}: {materials}")
        else:
            print(f"⚪ {cid}: No materials")
    except Exception as e:
        print(f"❌ {cid}: Error - {e}")
    time.sleep(0.5)  # be polite to the server

# Flatten and save to CSV
rows = []
for item in component_data:
    for mat in item["materials"]:
        rows.append({"component_id": item["component_id"], "material": mat})

df = pd.DataFrame(rows)
df

⚪ 558048: No materials
⚪ 558049: No materials


""


In [ ]:
import requests
from bs4 import BeautifulSoup

# Replace with the name/value from your browser exactly
COOKIE_NAME = "sid"
COOKIE_VALUE = "17f98ed892c219e814ccc434836ee213"

def fetch_component_materials(component_id: int):
    url = f"https://www.bauteileditor.de/elements/general/?e={component_id}&tab=general"
    
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Cookie": f"sid={COOKIE_VALUE}"  # replace key name if it's different
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract materials from HTML structure
    materials = []
    for li in soup.select("#element-composite ol.sortable li .element-selector"):
        materials.append(li.text.strip())

    return materials

# Test it
component_id = 558048
materials = fetch_component_materials(component_id)
print(f"Component {component_id} materials:", materials)



Component 558048 materials: []


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 💡 1. Set your session cookie (grabbed from browser)
COOKIE_VALUE = "17f98ed892c219e814ccc434836ee213"

# 💡 2. Get UUID from a material's detail page
def get_material_uuid(material_id):
    url = f"https://www.bauteileditor.de/elements/general/?e={material_id}&tab=general"
    headers = {"User-Agent": "Mozilla/5.0"}
    cookies = {"sid": COOKIE_VALUE}

    try:
        response = requests.get(url, headers=headers, cookies=cookies, timeout=10)
        if response.status_code != 200:
            return None

        soup = BeautifulSoup(response.text, "html.parser")
        uuid_elem = soup.find("span", class_="selection-value", string=lambda x: x and "-" in x)
        return uuid_elem.text.strip() if uuid_elem else None
    except Exception as e:
        print(f"UUID fetch failed for {material_id}: {e}")
        return None

# 💡 3. For each component, get its name and linked materials
def fetch_component_data(component_id):
    url = f"https://www.bauteileditor.de/elements/general/?e={component_id}&tab=general"
    headers = {"User-Agent": "Mozilla/5.0"}
    cookies = {"sid": COOKIE_VALUE}

    try:
        response = requests.get(url, headers=headers, cookies=cookies, timeout=10)
        if response.status_code != 200:
            return []

        soup = BeautifulSoup(response.text, "html.parser")

        # Component name (e.g., "Außenwand / einschaliges Mauerwerk / erdberührt")
        name_input = soup.select_one("input[name='name']")
        component_name = name_input["value"] if name_input else None

        materials_block = soup.select("#element-composite ol.sortable li")
        results = []

        for li in materials_block:
            name_tag = li.select_one(".element-selector")
            link_tag = li.select_one("a[href*='/elements/'][href*='tab=general']")

            if name_tag and link_tag:
                material_name = name_tag.text.strip()
                material_id = link_tag["href"].split("/")[2]  # e.g., /elements/558049/
                material_uuid = get_material_uuid(material_id)

                results.append({
                    "component_id": component_id,
                    "component_name": component_name,
                    "material_name": material_name,
                    "material_id": material_id,
                    "material_uuid": material_uuid
                })

        return results

    except Exception as e:
        print(f"❌ Component {component_id} failed: {e}")
        return []

# 💡 4. Loop over component IDs
all_data = []
start_id = 1900
end_id = 1910  # expand this when stable

for cid in range(start_id, end_id):
    materials_info = fetch_component_data(cid)
    if materials_info:
        all_data.extend(materials_info)
        print(f"✅ {cid} → {len(materials_info)} materials")
    else:
        print(f"— {cid} skipped")
    time.sleep(0.25)  # delay to avoid hammering the server

# 💡 5. Save results
df = pd.DataFrame(all_data)
df.to_csv("component_materials_with_uuids.csv", index=False)
print("📦 Saved to component_materials_with_uuids.csv")


— 1900 skipped
— 1901 skipped
— 1902 skipped
— 1903 skipped
— 1904 skipped
— 1905 skipped
— 1906 skipped
— 1907 skipped
✅ 1908 → 3 materials
— 1909 skipped
📦 Saved to component_materials_with_uuids.csv


## Script to Scrape Category Overview Page
Here's an example script to scrape all components from one category (e.g. 330 exterior walls, t=246)
The URL you're referring to is a client-side rendered (JavaScript) app (with a #!/ in the path), which means that the content (like the component list) is not available in the raw HTML of the page source fetched with requests. That’s why BeautifulSoup is returning an empty result. We will use selenium instead

In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

# Setup headless Chrome
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

def fetch_component_name_and_id_selenium(url):
    driver.get(url)
    time.sleep(5)  # wait for JS to load

    soup = BeautifulSoup(driver.page_source, "html.parser")

    components = []

    for h2 in soup.select("h2.headline.ref-element"):
        text = h2.get_text(strip=True)
        match = re.match(r"(.*)\[(\d+)\]$", text)
        if match:
            name, comp_id = match.groups()
            components.append({
                "component_id": int(comp_id),
                "component_name": name.strip()
            })

    return pd.DataFrame(components)

# Run
url = "https://www.bauteileditor.de/elements/#!/elements/list/?t=246"
df = fetch_component_name_and_id_selenium(url)
print(df)

driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Set Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 👇 Correct way to provide the ChromeDriver path
service = Service("/usr/local/bin/chromedriver")

# Create driver with service and options
driver = webdriver.Chrome(service=service, options=options)

# Test run
driver.get("https://www.google.com")
print(driver.title)

driver.quit()




Google


In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import time

# === STEP 1: Setup Chrome ===
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service("/usr/local/bin/chromedriver")

driver = webdriver.Chrome(service=service, options=options)

# === STEP 2: Visit base domain to enable cookie injection ===
driver.get("https://www.bauteileditor.de")
time.sleep(1)

# === STEP 3: Add your session cookie ===

driver.add_cookie({
    'name': 'sid',
    'value': "17f98ed892c219e814ccc434836ee213",
    'domain': 'www.bauteileditor.de',
    'path': '/',
    'secure': True,
    'httpOnly': True,
    'sameSite': 'Strict'
})

# === STEP 4: Now visit the protected page ===
driver.get("https://www.bauteileditor.de/elements/#!/elements/list/?t=246")
time.sleep(5)

# === STEP 5: Scroll to load content ===
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# === STEP 6: Extract component names and IDs ===
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "ref-element"))
)

elements = driver.find_elements(By.CLASS_NAME, "ref-element")

component_data = []
for el in elements:
    text = el.text.strip()
    match = re.search(r"(.*)\[(\d+)\]", text)
    if match:
        name = match.group(1).strip()
        comp_id = int(match.group(2))
        component_data.append({"id": comp_id, "name": name})

print(driver.current_url)
print(driver.title)
print(driver.page_source[:1000])  # Show the first 1000 chars of the HTML


# === STEP 7: Convert to DataFrame ===
df = pd.DataFrame(component_data)
print(df.head())
driver.quit()


TimeoutException: Message: 


In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re
import pandas as pd
import time

options = Options()
options.add_argument("--headless=new")  # ✅ New headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# === STEP 2: Visit base domain to enable cookie injection ===
driver.get("https://www.bauteileditor.de")
time.sleep(1)

# === STEP 3: Add your session cookie ===

driver.add_cookie({
    'name': 'sid',
    'value': "17f98ed892c219e814ccc434836ee213",
    'domain': 'www.bauteileditor.de',
    'path': '/',
    'secure': True,
    'httpOnly': True,
    'sameSite': 'Strict'
})

# ✅ Use the real, rendered list page
url = "https://www.bauteileditor.de/elements/list/?t=246"
driver.get(url)

# Allow time for page to fully load
time.sleep(5)

# Scroll slowly in case more content loads dynamically
for _ in range(3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1.5)

# ✅ Extract all component name/id headers
elements = driver.find_elements(By.CSS_SELECTOR, "h2.headline.ref-element")

component_data = []
for el in elements:
    text = el.text.strip()
    match = re.search(r"(.*?)\s*\[(\d+)\]", text)
    if match:
        name = match.group(1).strip()
        comp_id = int(match.group(2))
        component_data.append({"id": comp_id, "name": name})

driver.quit()

# Output result
df = pd.DataFrame(component_data)
print(df.head())


Empty DataFrame
Columns: []
Index: []


In [65]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re
import pandas as pd
import time

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# STEP 1: Open stub page to enable cookie injection
driver.get("https://www.bauteileditor.de/favicon.ico")
time.sleep(1)

# STEP 2: Inject cookie
driver.add_cookie({
    'name': 'sid',
    'value': "17f98ed892c219e814ccc434836ee213",  # Your real SID
    'domain': 'www.bauteileditor.de',
    'path': '/',
})

# STEP 3: Now go to the real page
url = "https://www.bauteileditor.de/elements/list/?t=246"
driver.get(url)
time.sleep(5)

# STEP 4: Scroll and wait
for _ in range(3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# STEP 5: Extract components
elements = driver.find_elements(By.CSS_SELECTOR, "h2.headline.ref-element")

component_data = []
for el in elements:
    text = el.text.strip()
    match = re.search(r"(.*?)\s*\[(\d+)\]", text)
    if match:
        name = match.group(1).strip()
        comp_id = int(match.group(2))
        component_data.append({"id": comp_id, "name": name})

driver.quit()

# STEP 6: Output
df = pd.DataFrame(component_data)
# to csv
df.to_csv("components_list.csv", index=False)



In [66]:
df

,id,name
0,558048,Außenwand / einschaliges Mauerwerk / erdberührt
1,1908,Außenwand / einschaliges Mauerwerk / Innendämmung
2,450096,Außenwand / einschaliges Mauerwerk / Innendämmung
3,1955,Außenwand / einschaliges Mauerwerk / WDVS / In...
4,13612,Außenwand / einschaliges Mauerwerk / WDVS mit ...
5,450081,Außenwand / einschaliges Mauerwerk / WDVS mit ...
6,11547,Außenwand / einschaliges Mauerwerk / WDVS / ve...
7,43850,Außenwand Gaube / Holzrahmenbau / Schieferverk...
8,450092,Außenwand Gaube / Holzrahmenbau / Schieferverk...
9,556553,Außenwand / Holzmassivbau / Holzverkleidung


In [63]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# === Configuration ===
COOKIE_VALUE = "17f98ed892c219e814ccc434836ee213"
HEADERS = {"User-Agent": "Mozilla/5.0"}
COOKIES = {"sid": COOKIE_VALUE}

# ✅ 1. Get UUID from material detail page
def get_material_uuid(material_id):
    url = f"https://www.bauteileditor.de/elements/general/?e={material_id}&tab=general"
    try:
        print(f"🔍 Fetching UUID for material ID {material_id}...")
        response = requests.get(url, headers=HEADERS, cookies=COOKIES, timeout=10)
        print(f"Status Code: {response.status_code}")
        if response.status_code != 200:
            return None
        soup = BeautifulSoup(response.text, "html.parser")
        uuid_elem = soup.find("span", class_="selection-value", string=lambda x: x and "-" in x)
        return uuid_elem.text.strip() if uuid_elem else None
    except Exception as e:
        print(f"UUID fetch failed for {material_id}: {e}")
        return None

# ✅ 2. Get materials for a component ID
def fetch_component_data(component_id):
    url = f"https://www.bauteileditor.de/elements/general/?e={component_id}&tab=general"
    try:
        print(f"\n🧱 Fetching component ID {component_id}...")
        response = requests.get(url, headers=HEADERS, cookies=COOKIES, timeout=10)
        print(f"Component status code: {response.status_code}")
        if response.status_code != 200:
            return []

        soup = BeautifulSoup(response.text, "html.parser")
        name_input = soup.select_one("input[name='name']")
        component_name = name_input["value"] if name_input else None
        print(f"Component name: {component_name}")

        materials_block = soup.select("#element-composite ol.sortable li")
        results = []

        for li in materials_block:
            name_tag = li.select_one(".element-selector")
            link_tag = li.select_one("a[href*='/elements/'][href*='tab=general']")

            if name_tag and link_tag:
                material_name = name_tag.text.strip()
                material_id = link_tag["href"].split("/")[2]
                material_uuid = get_material_uuid(material_id)

                results.append({
                    "component_id": component_id,
                    "component_name": component_name,
                    "material_name": material_name,
                    "material_id": material_id,
                    "material_uuid": material_uuid
                })

        return results
    except Exception as e:
        print(f"❌ Component {component_id} failed: {e}")
        return []

# ✅ 3. Test with a single known-good ID
test_component_ids = [558048]  # Außenwand / einschaliges Mauerwerk / erdberührt

all_data = []
for cid in test_component_ids:
    materials_info = fetch_component_data(cid)
    if materials_info:
        all_data.extend(materials_info)
        print(f"✅ {cid} → {len(materials_info)} materials found.")
    else:
        print(f"— {cid} skipped or empty.")
    time.sleep(0.25)

# ✅ 4. Output results
df = pd.DataFrame(all_data)



🧱 Fetching component ID 558048...
Component status code: 200
Component name: Außenwand / einschaliges Mauerwerk / erdberührt
🔍 Fetching UUID for material ID 558049...
Status Code: 200
🔍 Fetching UUID for material ID 558050...
Status Code: 200
🔍 Fetching UUID for material ID 558051...
Status Code: 200
✅ 558048 → 3 materials found.


In [64]:
df

,component_id,component_name,material_name,material_id,material_uuid
0,558048,Außenwand / einschaliges Mauerwerk / erdberührt,Kalk Innenputz,558049,987fd96c-daad-4f08-8e26-a1dfac76b505
1,558048,Außenwand / einschaliges Mauerwerk / erdberührt,Kalksandstein,558050,71d49937-5585-4dea-ba1f-456dbb70e338
2,558048,Außenwand / einschaliges Mauerwerk / erdberührt,Dämmung - Abdichtung,558051,95b680d4-7813-4c4b-8e36-666230a0f5f3


In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# === Configuration ===
COOKIE_VALUE = "17f98ed892c219e814ccc434836ee213"
HEADERS = {"User-Agent": "Mozilla/5.0"}
COOKIES = {"sid": COOKIE_VALUE}

# ✅ Load list of component IDs and names (from your Selenium scrape)
components_df = pd.read_csv("components_list.csv")
print(f"Loaded {len(components_df)} components.")

# ✅ 1. Get UUID from material detail page
def get_material_uuid(material_id):
    url = f"https://www.bauteileditor.de/elements/general/?e={material_id}&tab=general"
    try:
        response = requests.get(url, headers=HEADERS, cookies=COOKIES, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        uuid_elem = soup.find("span", class_="selection-value", string=lambda x: x and "-" in x)
        return uuid_elem.text.strip() if uuid_elem else None
    except Exception as e:
        print(f"UUID fetch failed for {material_id}: {e}")
        return None

# ✅ 2. Get materials for a component ID
def fetch_component_data(component_id, component_name):
    url = f"https://www.bauteileditor.de/elements/general/?e={component_id}&tab=general"
    try:
        response = requests.get(url, headers=HEADERS, cookies=COOKIES, timeout=10)
        if response.status_code != 200:
            return []

        soup = BeautifulSoup(response.text, "html.parser")
        materials_block = soup.select("#element-composite ol.sortable li")
        results = []

        for li in materials_block:
            name_tag = li.select_one(".element-selector")
            link_tag = li.select_one("a[href*='/elements/'][href*='tab=general']")

            if name_tag and link_tag:
                material_name = name_tag.text.strip()
                material_id = link_tag["href"].split("/")[2]
                material_uuid = get_material_uuid(material_id)

                results.append({
                    "component_id": component_id,
                    "component_name": component_name,
                    "material_name": material_name,
                    "material_id": material_id,
                    "material_uuid": material_uuid
                })

        return results
    except Exception as e:
        print(f"❌ Component {component_id} failed: {e}")
        return []

# ✅ 3. Loop over all components
all_results = []

for idx, row in components_df.iterrows():
    cid = row["id"]
    cname = row["name"]
    print(f"🔄 Processing {cid} - {cname}")
    data = fetch_component_data(cid, cname)
    if data:
        all_results.extend(data)
        print(f"✅ Found {len(data)} materials.")
    else:
        print(f"— Skipped or empty.")
    time.sleep(0.3)

# ✅ 4. Save to CSV
df_final = pd.DataFrame(all_results)
df_final.to_csv("component_materials_with_uuids.csv", index=False)
print("🎉 Done! Data saved to component_materials_with_uuids.csv")  


Loaded 24 components.
🔄 Processing 558048 - Außenwand / einschaliges Mauerwerk / erdberührt
✅ Found 3 materials.
🔄 Processing 1908 - Außenwand / einschaliges Mauerwerk / Innendämmung
✅ Found 3 materials.
🔄 Processing 450096 - Außenwand / einschaliges Mauerwerk / Innendämmung
✅ Found 3 materials.
🔄 Processing 1955 - Außenwand / einschaliges Mauerwerk / WDVS / Innenfliesen
✅ Found 3 materials.
🔄 Processing 13612 - Außenwand / einschaliges Mauerwerk / WDVS mit Fenster
✅ Found 3 materials.
🔄 Processing 450081 - Außenwand / einschaliges Mauerwerk / WDVS mit Fenster
✅ Found 3 materials.
🔄 Processing 11547 - Außenwand / einschaliges Mauerwerk / WDVS / verputzt
✅ Found 3 materials.
🔄 Processing 43850 - Außenwand Gaube / Holzrahmenbau / Schieferverkleidung
✅ Found 3 materials.
🔄 Processing 450092 - Außenwand Gaube / Holzrahmenbau / Schieferverkleidung
✅ Found 3 materials.
🔄 Processing 556553 - Außenwand / Holzmassivbau / Holzverkleidung
✅ Found 3 materials.
🔄 Processing 558275 - Außenwand / Hol

In [68]:
df_final

,component_id,component_name,material_name,material_id,material_uuid
0,558048,Außenwand / einschaliges Mauerwerk / erdberührt,Kalk Innenputz,558049,987fd96c-daad-4f08-8e26-a1dfac76b505
1,558048,Außenwand / einschaliges Mauerwerk / erdberührt,Kalksandstein,558050,71d49937-5585-4dea-ba1f-456dbb70e338
2,558048,Außenwand / einschaliges Mauerwerk / erdberührt,Dämmung - Abdichtung,558051,95b680d4-7813-4c4b-8e36-666230a0f5f3
3,1908,Außenwand / einschaliges Mauerwerk / Innendämmung,Porenbeton-Dämmplatte_25mm_geklebt_Kalkputz_E,3277,6398a647-6b33-4dec-b8e8-db63832bd85e
4,1908,Außenwand / einschaliges Mauerwerk / Innendämmung,Mauerziegel_270mm_E,1876,2da50076-2478-4b48-ab0c-ee00a1012be4
...,...,...,...,...,...
62,11302,Außenwand / zweischaliges Mauerwerk / Kerndämmung,Kalksandstein 24cm,1810,549291bf-6a21-4816-a3a1-a01a1889949b
63,11302,Außenwand / zweischaliges Mauerwerk / Kerndämmung,MW / Kerndämmung,11303,418d58f5-3112-4cf4-b179-9aba3efa6f37
64,450117,Außenwand / zweischaliges Mauerwerk / Kerndämmung,Gips-Putz/Anstrich,450118,8b676bc4-0747-4319-b64d-9a8008be4ad1
65,450117,Außenwand / zweischaliges Mauerwerk / Kerndämmung,Kalksandstein 24cm,450119,f05dd3e3-21c1-4e32-b352-f5d3a8fe2f64


In [71]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# === Setup Selenium ===
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# === Session cookie ===
SESSION_COOKIE = "17f98ed892c219e814ccc434836ee213"
driver.get("https://www.bauteileditor.de/favicon.ico")
time.sleep(1)
driver.add_cookie({
    'name': 'sid',
    'value': SESSION_COOKIE,
    'domain': 'www.bauteileditor.de',
    'path': '/',
})

# === HTTP settings for requests ===
HEADERS = {"User-Agent": "Mozilla/5.0"}
COOKIES = {"sid": SESSION_COOKIE}

# === Extract UUID from material detail page ===
def get_material_uuid(material_id):
    url = f"https://www.bauteileditor.de/elements/general/?e={material_id}&tab=general"
    try:
        r = requests.get(url, headers=HEADERS, cookies=COOKIES, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        uuid_elem = soup.find("span", class_="selection-value", string=lambda x: x and "-" in x)
        return uuid_elem.text.strip() if uuid_elem else None
    except:
        return None

# === Fetch component's materials and UUIDs ===
def fetch_component_materials(component_id):
    url = f"https://www.bauteileditor.de/elements/general/?e={component_id}&tab=general"
    try:
        r = requests.get(url, headers=HEADERS, cookies=COOKIES, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        name_input = soup.select_one("input[name='name']")
        component_name = name_input["value"] if name_input else None
        materials_block = soup.select("#element-composite ol.sortable li")

        results = []
        for li in materials_block:
            name_tag = li.select_one(".element-selector")
            link_tag = li.select_one("a[href*='/elements/'][href*='tab=general']")
            if name_tag and link_tag:
                material_name = name_tag.text.strip()
                material_id = link_tag["href"].split("/")[2]
                material_uuid = get_material_uuid(material_id)
                results.append({
                    "component_id": component_id,
                    "component_name": component_name,
                    "material_name": material_name,
                    "material_id": material_id,
                    "material_uuid": material_uuid
                })
        return results
    except:
        return []

# === Main categories to loop over ===
main_categories = [237, 246, 256, 264]  # skipping 269 and 270 as per user

# === Scrape all components and their materials ===
all_data = []

for t in main_categories:
    print(f"\n🔄 Scraping main category t={t}")
    url = f"https://www.bauteileditor.de/elements/list/?t={t}"
    driver.get(url)
    time.sleep(5)

    # Scroll to load full page content
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    elements = driver.find_elements(By.CSS_SELECTOR, "h2.headline.ref-element")
    component_ids = []
    for el in elements:
        match = re.search(r"(.*?)\s*\[(\d+)\]", el.text.strip())
        if match:
            comp_id = int(match.group(2))
            component_ids.append(comp_id)

    print(f"✅ Found {len(component_ids)} components")

    for cid in component_ids:
        materials = fetch_component_materials(cid)
        if materials:
            all_data.extend(materials)
            print(f"   ✅ {cid}: {len(materials)} materials")
        else:
            print(f"   — {cid}: no materials or skipped")
        time.sleep(0.25)

driver.quit()

# Save and display
df_all = pd.DataFrame(all_data)
# df_all.to_csv("all_components_with_materials.csv", index=False)




🔄 Scraping main category t=237
✅ Found 12 components
   ✅ 11217: 2 materials
   ✅ 43836: 2 materials
   ✅ 450110: 2 materials
   ✅ 1124879: 2 materials
   ✅ 11218: 2 materials
   ✅ 11221: 2 materials
   ✅ 2780: 3 materials
   ✅ 450113: 3 materials
   ✅ 1124883: 3 materials
   ✅ 3632: 4 materials
   ✅ 450101: 3 materials
   ✅ 1124887: 3 materials

🔄 Scraping main category t=246
✅ Found 24 components
   ✅ 558048: 3 materials
   ✅ 1908: 3 materials
   ✅ 450096: 3 materials
   ✅ 1955: 3 materials
   ✅ 13612: 3 materials
   ✅ 450081: 3 materials
   ✅ 11547: 3 materials
   ✅ 43850: 3 materials
   ✅ 450092: 3 materials
   ✅ 556553: 3 materials
   ✅ 558275: 3 materials
   ✅ 11224: 3 materials
   ✅ 450088: 3 materials
   ✅ 11230: 3 materials
   ✅ 450529: 3 materials
   ✅ 558532: 2 materials
   ✅ 11234: 2 materials
   ✅ 11238: 2 materials
   ✅ 11318: 2 materials
   ✅ 450509: 2 materials
   ✅ 11243: 3 materials
   ✅ 450503: 3 materials
   ✅ 11302: 3 materials
   ✅ 450117: 3 materials

🔄 Scraping

In [72]:
df_all

,component_id,component_name,material_name,material_id,material_uuid
0,11217,Bodenplatte / oberseitig gedämmt / Estrich / L...,oberseitige Dämmung / Estrich / Linolium,11216,6abf6a5a-73bb-40ba-877f-40d912e72c03
1,11217,Bodenplatte / oberseitig gedämmt / Estrich / L...,Fundamentplatte Stb. / Dränschicht,11211,1d5c3920-94c8-4b1f-ac78-50462ad483ba
2,43836,Bodenplatte / oberseitig gedämmt / Sand / schw...,G_Voranstrich_Kunstharz_5mm,43837,f9d4dfc9-7e91-4625-9963-a1cd732024e6
3,43836,Bodenplatte / oberseitig gedämmt / Sand / schw...,G_Bodenplatte_Stahlbeton-Ortbauweise_C25/30_25...,43844,20a90bce-17e1-40f8-9a22-db8adb923e29
4,450110,Bodenplatte / oberseitig gedämmt / Sand / schw...,G_Voranstrich_Kunstharz_5mm,450111,12306b63-5e9d-403a-9357-ffd80eeb960d
...,...,...,...,...,...
168,497538,Kellerdecke / Stb. / unterseitig gedämmt / sch...,IBO_GDm_01,497540,6a716ece-8aff-4b9e-91d5-f790db667192
169,497538,Kellerdecke / Stb. / unterseitig gedämmt / sch...,IBO_KDb_01,497541,4d1dc398-8995-475e-bc60-a554b1256a65
170,4644674,Kellerdecke / Stb. / unterseitig gedämmt / sch...,IBO_KDb_01,4644675,ed35c313-0396-49c5-940e-7aefa3dd5175
171,4644674,Kellerdecke / Stb. / unterseitig gedämmt / sch...,IBO_GDm_01,4644676,bb2ae572-4151-40e3-9a25-651edd124e27
